  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5.2
  Using cached keras_rl2-1.0.5-py3-none-any.whl (52 kB)
Note: you may need to restart the kernel to use updated packages.


  Using cached tensorboard-2.1.1-py3-none-any.whl (3.8 MB)
  Using cached tensorflow_estimator-2.1.0-py2.py3-none-any.whl (448 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Can't uninstall 'tensorflow-estimator'. No files were found to uninstall.
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.0
    Can't uninstall 'tensorboard'. No files were found to uninstall.
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Can't uninstall 'scipy'. No files were found to uninstall.


In [60]:
import gym
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [65]:
class Shower(Env):
    def __init__(self):
        #for actions down, stay, up
        self.action_space = Discrete(3)
        #temperature array for range
        self.observation_space = Box(low = np.array([0]), high = np.array([100]))
        #setting starting temperature
        self.state = 38 + random.randint(-3, 3)
        #set shower length
        self.shower_length = 60
        
    
    
        
    def step(self, action):
        #applying action , 0 => we minus 1 to perform decrease temp by 1 
        #1 => we minus 0, to stay at same temp
        #2 => we add 1 to increase temp
        self.state += action - 1
        #reduce shower length by 1
        self.shower_length -=1
        
        #Calculate reward
        if self.state >=37 and self.state <= 39:
            reward = 10
        else:
            reward = -1
            
        #check done
        if self.shower_length <=0:
            done = True
        else:
            done = False
            
        #temp noise
#         self.state += random.randint(-.5, .5)
        
        #define info
        info = {}
        
        #step info
        return self.state, reward, done, info
    
    
    def reset(self):
        #reset temperature
        self.state = 38 + random.randint(-3, 3)
        
        #reset shower time
        self.shower_length = 60
        return self.state

In [66]:
env = Shower()

In [67]:
episodes = 10
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
    

Episode:0 Score:39
Episode:1 Score:-16
Episode:2 Score:6
Episode:3 Score:413
Episode:4 Score:116
Episode:5 Score:-27
Episode:6 Score:281
Episode:7 Score:94
Episode:8 Score:-49
Episode:9 Score:402


In [68]:
import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam

In [69]:
states = env.observation_space.shape
actions = env.action_space.n


In [70]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation = 'relu', input_shape = states))        
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model

In [71]:
del model

In [72]:
model = build_model(states, actions)

In [73]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 24)                48        
_________________________________________________________________
dense_11 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [74]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [75]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [76]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

AttributeError: 'tuple' object has no attribute 'layer'